<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/HSS__TrainSet_Cleaning_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules.

In [1]:
import os
import os.path
from os import path
import re
import shutil
import string
import tensorflow as tf

!pip install -U deep-translator
from deep_translator import GoogleTranslator
from io import open
from pathlib import Path
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 5.2 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
!pip install xmltodict
import xmltodict 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing DS and extract in current working directory.

In [3]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
train_set_url = "https://github.com/marco-siino/DA-ESWA/blob/main/data/hss/hss-training-original.zip?raw=true"
train_set_path = tf.keras.utils.get_file("pan21-author-profiling-training-2021-03-14.zip", train_set_url,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

train_set_dir = os.path.join(os.path.dirname(train_set_path), 'pan21-author-profiling-training-2021-03-14.zip')

print(train_set_path)
print(train_set_dir)

2822571/2822571 [==============================] - 0s 0us/step
./pan21-author-profiling-training-2021-03-14.zip
./pan21-author-profiling-training-2021-03-14.zip


## Function to pre-process source text.

In [4]:
def custom_standardization(text):
  tag_open_CDATA_removed = re.sub('<\!\[CDATA\[', ' ',text)
  tag_closed_CDATA_removed = re.sub('\]{1,}>', ' ',tag_open_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en">', ' ',tag_closed_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en"', ' ',tag_closed_CDATA_removed)
  #only for test remove if needed
  tag_author_lang_en_removed = re.sub('class="0">', ' ', tag_author_lang_en_removed )
  tag_author_lang_en_removed = re.sub('class="1">', ' ', tag_author_lang_en_removed )
  ######
  tag_closed_author_removed = re.sub('</author>', '', tag_author_lang_en_removed)
  tag_closed_author_removed = re.sub('</author', '', tag_closed_author_removed)
  tag_open_documents_removed = re.sub('<documents>\n(\t){0,2}', '',tag_closed_author_removed)
  tag_closed_documents_removed = re.sub('</documents>\n(\t){0,2}', ' ',tag_open_documents_removed)
  lowercased_text = tag_closed_documents_removed.lower()
  return lowercased_text

## Data Augmentation function.

In [5]:
def chunkstring(string, length):
  res = list((string[0+i:length+i] for i in range(0, len(string), length)))
  return res

def enhance_one_sample(sample, TARGET='it', return_both=True):  
  preprocessed_text = custom_standardization(sample)

  #chunk to avoid character limits  
  TOBETRANS = chunkstring(preprocessed_text, 4000)
  translated_it = GoogleTranslator(source='en', target=TARGET).translate_batch(TOBETRANS)
  reversed_trans = GoogleTranslator(source=TARGET, target='en').translate_batch(translated_it)
  merged_chunks =''.join(reversed_trans)
  enhanced_sample = preprocessed_text+merged_chunks
  if return_both == False:
    enhanced_sample = merged_chunks
  return enhanced_sample

# Generating augmented train set.

In [6]:
## Generate the augmented training set as a zip file.

# pan22-author-profiling-training-2022-03-29-augmented.zip has to be created.
if not os.path.exists('pan21-author-profiling-training-2021-03-14-augmented'):
    os.makedirs('pan21-author-profiling-training-2021-03-14-augmented')
if not os.path.exists('pan21-author-profiling-training-2021-03-14-augmented/en/'):
    os.makedirs('pan21-author-profiling-training-2021-03-14-augmented/en/')

In [15]:
# Copy the ground truth file.
if os.path.exists('/content/pan21-author-profiling-training-2021-03-14/en'):
        shutil.copyfile('/content/pan21-author-profiling-training-2021-03-14/en/truth.txt', '/content/pan21-author-profiling-training-2021-03-14-augmented/en/truth.txt')

In [16]:
folder_path = "/content/pan21-author-profiling-training-2021-03-14/en/"
output_path = '/content/pan21-author-profiling-training-2021-03-14-augmented/en/'
#output_file = "/content/pan21-author-profiling-test-without-gold-augmented/truth.txt"
for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
        # Open the XML file
        print('##########')
        print(filename)
        text = open('pan21-author-profiling-training-2021-03-14/en/'+filename, 'r').read()
        enhanced_sample = custom_standardization(text)
        #print(enhanced_sample)
        x = filename.split(".")      
        author_id = x[0]
        f = open( output_path + author_id+".xml", "a")
        f.write(enhanced_sample)
        f.close()
# with open(output_file, "a") as out:
#   # Loop over all files in the folder
#   for filename in os.listdir(folder_path):
#       if filename.endswith(".xml"):
#           # Open the XML file
#           print('##########')
#           x = filename.split(".")      
#           author_id = x[0]
#           with open(os.path.join(folder_path, filename)) as xml_file:
#               # Parse the XML file into an ordered dictionary
#               xml_dict = xmltodict.parse(xml_file.read())
#               # Print all keys of the file
#               print(filename)
#               print("xml_dict['author']['documents']", xml_dict['author']['documents'])
#               for key in xml_dict.keys():
#                   #print([*xml_dict['author']])
#                   #print(xml_dict['author']['@class'])
#                   CLASS = xml_dict['author']['@class']
#                   STR = filename.split('.')[0] + ':::' + CLASS + '\n'
#               out.write(STR)  
# out.close()

!zip -r pan21-author-profiling-training-2021-03-14-augmented.zip pan21-author-profiling-training-2021-03-14-augmented
# If automatic download doesn't start, open the directory browser on the left menu and download the zip file manually.
files.download("pan21-author-profiling-training-2021-03-14-augmented.zip")

##########
5b50fcecc61c850e961ff307bdce67a0.xml
##########
4ae4ddc8cb2774c92398e3102c3da5b2.xml
##########
27e3e134eb8a0069e220bc201cda8eb9.xml
##########
d2e0f4f0244b9b8b3bbd8b1654be5b74.xml
##########
afe04507770754d25fcdaa901841d440.xml
##########
b7f4ead8c975203082dcea622621d2c3.xml
##########
94b5cb02328011bbecaf78d79d2c1ff5.xml
##########
dead14502e55f545546f666ab6d91558.xml
##########
365eb1e3abc5cd5394fec8fc162bfbc5.xml
##########
2a9776a2909ba5e27043faf2d57ba410.xml
##########
46dc7954a1dae6630bef96513f4684ec.xml
##########
d0c3083a5ff95c520df956229dbd4423.xml
##########
135cc1b889864768b5c755eb6210d358.xml
##########
f30abea44b6c4144c0690f98459428a6.xml
##########
a887ec85088a87e550015e2770a6e309.xml
##########
b496caf332cb0ba97d2acefc44f153ac.xml
##########
43641af5b9182e88190638f972eb97eb.xml
##########
e5cc453c1de4371b8d668c3ab50126f1.xml
##########
8b7608a503048538181399629fe67913.xml
##########
76e152a7732922e7a6da39880486107f.xml
##########
9cff4936f8479d53fcbb63f2524c5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
file_nr = 0
count_errors = 0
for filename in os.listdir('pan20-author-profiling-training-2020-02-23/en/'):
  file_nr += 1
  x = filename.split(".")      
  author_id = x[0]
  print("File nr.:", file_nr)
  print("Filename:", filename)
  text = open('pan20-author-profiling-training-2020-02-23/en/'+filename, 'r').read()
  try:
    enhanced_sample = enhance_one_sample(text, TARGET='it')
    #de_back = enhance_one_sample(text, TARGET='de', return_both=False)
    #enhanced_sample = enhanced_sample + de_back    
    f = open("pan20-author-profiling-training-2020-02-23-augmented/en/"+author_id+".xml", "a")
    f.write(enhanced_sample)
    f.close()
    print('Succes')
  except:
    print('!!! FAILED !!!!')
    count_errors += 1
    f = open("pan20-author-profiling-training-2020-02-23-augmented/en/"+author_id+".xml", "a")
    preprocessed_text = custom_standardization(text)
    f.write(preprocessed_text)
    f.close()

print('cunt err', count_errors)
!zip -r pan20-author-profiling-training-2020-02-23-augmented.zip pan20-author-profiling-training-2020-02-23-augmented
# If automatic download doesn't start, open the directory browser on the left menu and download the zip file manually.
files.download("pan20-author-profiling-training-2020-02-23-augmented.zip")

File nr.: 1
Filename: 9i4x0l6lzayvpooc4ybugncxxd58dvee.xml
Succes
File nr.: 2
Filename: iqx2tqavqtpmh1kdn5wnt2wvlvt4me97.xml
Succes
File nr.: 3
Filename: l4e0nfw0yzzaouhu9efltwdk6m95jloj.xml
Succes
File nr.: 4
Filename: j6wo3pt6m38ptcky7otksvpxwfmjm2sy.xml
Succes
File nr.: 5
Filename: 5we816p2ml0mujwg2ha1vicr0p2qtoo3.xml
Succes
File nr.: 6
Filename: x5xvdmiagqijl0uoe001wwekdfsw823b.xml
Succes
File nr.: 7
Filename: v4a5crla11s25l4qh3q6ttxtmdpqacak.xml
Succes
File nr.: 8
Filename: cz3s8ini8iyrb8uz8x4a32z8bnsmvol5.xml
Succes
File nr.: 9
Filename: a8w5juro2x4fbxo7bnwztfz0i8e61k9k.xml
Succes
File nr.: 10
Filename: dgm54nxhyr9cextksr1tnflxu416sama.xml
Succes
File nr.: 11
Filename: d6shpn9ebosxt4n1hhl64benshbezqnx.xml
Succes
File nr.: 12
Filename: nt0kallmiu3m93jvrwgll44wx6ooqnsa.xml
Succes
File nr.: 13
Filename: yx4q3zokksf3a3g3oapn055p1dnf0a91.xml
Succes
File nr.: 14
Filename: z6ejdyu1emh9z9mlsnt5km2suhgbonl9.xml
Succes
File nr.: 15
Filename: n3l9sc6sinb2up1qxytkppkaq2cu4hd0.xml
Succes
File

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>